In [8]:
# Make sure to pip install pyarrow and fastparquet
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('troop_movements_1m.csv')

print(data.head())

             timestamp  unit_id     unit_type  location_x  location_y  \
0  2023-06-03 03:19:15   919214  tie_silencer         2.0         5.0   
1  2023-02-24 13:50:40  9467154  stormtrooper         9.0         0.0   
2  2023-03-29 19:54:55  6585778  tie_silencer         0.0         6.0   
3  2023-04-30 00:58:11  3878023  tie_silencer         4.0         2.0   
4  2023-04-10 22:00:26  5537117         at-st         6.0         8.0   

   destination_x  destination_y    homeworld  
0              9              3  Aleen Minor  
1              9              1    Malastare  
2              5              9      Serenno  
3              9              9         Tund  
4              5              8        Skako  


In [3]:
data['unit_type'].str.replace('invalid_unit', 'unknown')

0         tie_silencer
1         stormtrooper
2         tie_silencer
3         tie_silencer
4                at-st
              ...     
999995     tie_fighter
999996           at-at
999997     tie_fighter
999998     tie_fighter
999999           at-st
Name: unit_type, Length: 1000000, dtype: object

In [4]:
data['location_x'] = data['location_x'].ffill()
data['location_y'] = data['location_y'].ffill()

In [5]:
data.to_parquet("troop_movements_1m.parquet")
data_loaded = pd.read_parquet("troop_movements_1m.parquet")

In [18]:
with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)

X = pd.get_dummies(data_loaded[['homeworld', 'unit_type']])

X = X.reindex(columns=model.feature_names_in_, fill_value=0)

data_loaded['predictions'] = model.predict(X)

In [21]:
data_loaded.tail(20)

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
999980,2023-03-29 04:29:31,6641529,at-at,5.0,7.0,5,6,Serenno,False
999981,2023-04-08 18:52:52,5721940,tie_fighter,6.0,0.0,1,1,Eriadu,False
999982,2023-03-13 05:10:35,8027248,x-wing,1.0,7.0,9,3,Tund,False
999983,2023-05-14 06:24:52,2643736,at-at,7.0,7.0,1,4,Skako,False
999984,2023-06-08 05:35:08,477144,tie_fighter,1.0,1.0,6,3,Concord Dawn,False
999985,2023-06-04 17:56:09,779577,x-wing,3.0,9.0,2,8,Rodia,False
999986,2023-06-04 05:16:18,825376,stormtrooper,7.0,7.0,5,5,Toydaria,False
999987,2023-03-03 20:11:52,8836894,stormtrooper,1.0,5.0,8,6,Muunilinst,False
999988,2023-02-23 13:41:08,9554483,at-st,8.0,1.0,5,8,Vulpter,False
999989,2023-03-14 16:26:45,7899755,tie_silencer,4.0,2.0,8,7,Quermia,False


In [22]:
filtered_df = data_loaded[data_loaded['predictions']]

print(filtered_df)

                  timestamp  unit_id           unit_type  location_x  \
31      2023-06-09 10:23:58   372966               at-at         4.0   
39      2023-03-29 15:00:32  6603513        tie_silencer         1.0   
46      2023-03-10 10:27:47  8264790              x-wing         5.0   
48      2023-04-15 07:57:03  5154149         tie_fighter         0.0   
58      2023-04-19 22:07:10  4755903        stormtrooper         9.0   
...                     ...      ...                 ...         ...   
999900  2023-02-23 23:40:15  9518389  resistance_soldier         2.0   
999923  2023-05-17 03:00:48  2395748              x-wing         5.0   
999928  2023-03-06 18:19:14  8583415        stormtrooper         7.0   
999954  2023-04-12 19:56:39  5371062               at-st         6.0   
999995  2023-05-03 18:48:56  3553240         tie_fighter         8.0   

        location_y  destination_x  destination_y  homeworld  predictions  
31             1.0              5              0   Dathomir 